# **SCRIPT PARA CAPTURAS DE DATA** 📸

In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [6]:
mp_holistic = mp.solutions.holistic # inicia el modelo donde podemos seguir los puntos del cuerpo humano, incluyendo pose, mano y caras.
mp_drawing = mp.solutions.drawing_utils # inicializa el dibujado o trazados de los puntos del modelo.
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh

In [7]:
def mediapipe_detection(image, model):
    """
    OpenCV carga las imágenes en el formato BGR por defecto. Sin embargo, 
    muchos modelos de procesamiento de imágenes, 
    incluido MediaPipe, esperan que las imágenes estén en formato RGB. 
    Por lo tanto, esta funcion convierte la imagen del formato BGR a RGB y devuelve la posición de
    los puntos  clave del cuerpo, la mano y la cara.
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [8]:
category = {
    'VOCALES': ['A','E','I','O','U']
}

## 1. CAPTURA UTILIZANDO MODELO HOLISTIC (MANOS, POSE Y CARA)

In [4]:
def draw_styled_landmarks(image,results):
    """
        La función permite conectar los puntes de referenica a la imagen y luego dibuja
        las conexiones, permitiendo ver (mallas).
    """
    # Dibuja los landmarks del cuerpo
    mp_drawing.draw_landmarks(image, #Toma como referencia la imagen capturada
                                  results.pose_landmarks, #Toma como referencia los puntos predecidos por el modelo Holistic.
                                  mp_pose.POSE_CONNECTIONS, #Conecta cada punto predecidos.
                                  mp_drawing.DrawingSpec( #Funcion que personaliza los puntos predecidos.
                                                         color=(80,22,10), #Color del punto
                                                         thickness=2, #Grosor del punto
                                                         circle_radius=4), #Radio del circulo.
                                  mp_drawing.DrawingSpec( #Funcion que personaliza las conexiones predecidas.
                                                         color=(80,44,121),
                                                         thickness=2,
                                                         circle_radius=2))
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image, results.face_landmarks,mp_face_mesh.FACEMESH_CONTOURS,mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [5]:
def extract_keypoints(results):
    face = np.array([[res.x,res.y,res.z]for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    rh = np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    pose = np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([pose,face,lh,rh])

In [ ]:
DATA_PATH = os.path.join('MP_Data_Webcam_Holistic')
no_videos = 30
no_frames = 30

In [ ]:
actions = np.array([value for values in category.values() for value in values])
actions

In [23]:
for action in actions:
    for no_video in range(no_videos):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(no_video)))
        except:
            pass

## 2. CAPTURA UTILIZANDO POSE  + MANOS

In [ ]:
def draw_styled_landmarks(image,results):
    """
        La función permite conectar los puntes de referenica a la imagen y luego dibuja
        las conexiones, permitiendo ver (mallas).
    """
    # Dibuja los landmarks del cuerpo
    mp_drawing.draw_landmarks(image, #Toma como referencia la imagen capturada
                                  results.pose_landmarks, #Toma como referencia los puntos predecidos por el modelo Holistic.
                                  mp_pose.POSE_CONNECTIONS, #Conecta cada punto predecidos.
                                  mp_drawing.DrawingSpec( #Funcion que personaliza los puntos predecidos.
                                                         color=(80,22,10), #Color del punto
                                                         thickness=2, #Grosor del punto
                                                         circle_radius=4), #Radio del circulo.
                                  mp_drawing.DrawingSpec( #Funcion que personaliza las conexiones predecidas.
                                                         color=(80,44,121),
                                                         thickness=2,
                                                         circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [ ]:
def extract_keypoints(results):
    rh = np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    pose = np.array([[res.x,res.y,res.z,res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([pose,lh,rh])

In [ ]:
DATA_PATH = os.path.join('MP_Data_Webcam_Pose_Manos')
no_videos = 30
no_frames = 30

In [ ]:
actions = np.array([value for values in category.values() for value in values])
actions

In [ ]:
for action in actions:
    for no_video in range(no_videos):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(no_video)))
        except:
            pass

## 3. CAPTURA UTILIZANDO SOLO MANOS

In [ ]:
def draw_styled_landmarks(image,results):
    """
        La función permite conectar los puntes de referenica a la imagen y luego dibuja
        las conexiones, permitiendo ver (mallas).
    """
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_hands.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

In [ ]:
def extract_keypoints(results):
    rh = np.array([[res.x,res.y,res.z]for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x,res.y,res.z]for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh,rh])

In [ ]:
DATA_PATH = os.path.join('MP_Data_Webcam_Manos')
no_videos = 30
no_frames = 30

In [ ]:
actions = np.array([value for values in category.values() for value in values])
actions

In [ ]:
for action in actions:
    for no_video in range(no_videos):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(no_video)))
        except:
            pass

## Captura Manual por cada seña.

In [29]:
cap = cv2.VideoCapture(0)
superBrake = False
sign = input('Escribe la seña a capturar: ')
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    """
        Aquí se está configurando el modelo Holistic de MediaPipe con una confianza mínima de detección de 0.5 y 
        una confianza mínima de seguimiento de 0.5. 
        El uso del with asegura que los recursos se liberen adecuadamente cuando se termina el bloque de código.
    """
    if sign in actions:
         while True:
             ret,image = cap.read()
             cv2.imshow('Recolectando Datos',image)
             if cv2.waitKey(10) & 0xFF == ord('q'):
                 break

         for no_video in range(no_videos): #Numero de video accion
            if superBrake:
                break
            cv2.waitKey(1000) #2000

            for frame_num in range(no_frames):#Numero de frame por video por accion
                # Read feed
                ret, frame = cap.read() #ret es solo un booleano que indica si la captura fue exitosa o no, frame es la imagen.

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                draw_styled_landmarks(image,results)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH,sign,str(no_video),str(frame_num))
                np.save(npy_path,keypoints)

                
                if no_video + 1 == no_videos and frame_num + 1 == no_frames:
                    cv2.putText(image,'PRESIONE CUALQUIERA TECLA PARA SALIR',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2,cv2.LINE_AA)
                    cv2.putText(image,'RECOLECTANDO VIDEO #{} PARA LA SENA <<{}>>'.format(no_video+1,sign),(15,35),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.putText(image,'FRAME CAPTURADO #{}'.format(frame_num+1),(15,60),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('Recolectando Datos',image)
                    cv2.waitKey(0)
                else: 
                    cv2.putText(image,'RECOLECTANDO VIDEO #{} PARA LA SENA <<{}>>'.format(no_video+1,sign),(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.putText(image,'FRAME CAPTURADO #{}'.format(frame_num+1),(15,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('Recolectando Datos',image)
                
                # cv2.imshow('Recolectando Datos',image)
                
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    superBrake = True
                    break

    else:
        print(f'La seña <{sign}> no se encuentra en el diccionario establecido para esta prueba.')

    cap.release()
    cv2.destroyAllWindows()

In [11]:
cap.release()
cv2.destroyAllWindows()